In [ ]:
%load_ext autoreload
%autoreload 1
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

import warnings

import dagster
warnings.filterwarnings("ignore", category=dagster.ExperimentalWarning)
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pandera as pa

from playground.util import resolve_data_path
import playground.repos.bollinger.lib as jpm

%aimport playground.repos.bollinger.lib


In [ ]:
PRICES = jpm.load_sp500_prices()

In [ ]:
BOLL = jpm.compute_bollinger_multi(PRICES)

In [ ]:
ANOMALIES = jpm.compute_anomalous_events(BOLL)

In [ ]:
plt.figure(figsize=(16,30))
for i,n in enumerate(BOLL.name.unique()[:5]):
    bdf = BOLL[BOLL.name == n]
    plt.subplot(5, 1, i+1)
    plt.title(n)
    plt.plot(bdf['price'])
    plt.plot(bdf['upper'])
    plt.plot(bdf['lower'])

In [ ]:
top_20 = (ANOMALIES.groupby('name').size()
    .sort_values(ascending=False)[:20].index.to_list())
plt.figure(figsize=(16,6))
df = ANOMALIES[ANOMALIES.name.isin(top_20)]
sns.stripplot(x='name', y='date', data=df, hue='type')

In [ ]:
# plt.figure(figsize=(16,16))
years = [pd.Timestamp(year, 1, 1) for year in range(2013,2019)]
plt.xticks(years)
targets = list(ANOMALIES.name.unique()[:2])
df = ANOMALIES[ANOMALIES.name.isin(targets)]
sns.stripplot(x='date', y='name', data=df)
# sns.stripplot(x='date', y='name', data=df, hue='type')